# Test AD Models 

Test other Anomaly Detection models to compare to EIF+. 

Potential models to test: 

- Deep Isolation Forest (DIF) 
- INNE: Isolation Based AD using nearest neighbours ensemble
- SUOD: This is another ensemble model like EIF 
- Auto Encoder 

In [1]:
import os
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
from pyod.models.dif import DIF
from pyod.models.inne import INNE
from pyod.models.auto_encoder import AutoEncoder
from pyod.models.suod import SUOD

from scipy.io import loadmat

sys.path.append('../src')
from src.performance_report_functions import *
from src.utils import *

sys.path.append('../models')
from models.forests import *

2024-02-09 16:24:48.980488: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE4.1 SSE4.2 AVX AVX2 AVX_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


## Load Data

In [2]:
os.chdir('../data/real/')
os.getcwd()

'/home/davidefrizzo/Desktop/PHD/ExIFFI/data/real'

In [3]:
X,y=mat_dataset('wine')

wine 

[number of samples = 129]
[percentage outliers = 0.07751937984496124]
[number features = 13]
[number outliers = 10]


In [6]:
X,y=csv_dataset('diabetes',os.getcwd()+'/')

diabetes 

[number of samples = 85916]
[percentage outliers = 0.096582708692211]
[number features = 4]
[number outliers = 8298]


In [4]:
X_train,X_test=partition_data(X,y)
X_train,X_test,X=pre_process(X_train,X_test)
X_train.shape,X_test.shape,X.shape

((119, 13), (129, 13), (129, 13))

## EIF

In [9]:
EIF=ExtendedIsolationForest(n_estimators=300,plus=0)
EIF.fit(X_train)

In [11]:
score_eif=EIF.predict(X_test)
perf=performance_eif(y,score_eif,X_test,EIF)
perf

{'Precision': 0.2,
 'Recall': 0.2,
 'f1 score': 0.20000000000000004,
 'Accuracy': 0.875968992248062,
 'Balanced Accuracy': 0.5663865546218487,
 'Average Precision': 0.1363386518400739,
 'ROC AUC Score': 0.6781512605042017}

## EIF+

In [12]:
EIF_plus=ExtendedIsolationForest(n_estimators=300,plus=1)
EIF_plus.fit(X_train)

In [13]:
score_eif_plus=EIF_plus.predict(X_test)
perf_plus=performance_eif(y,score_eif,X_test,EIF_plus)
perf_plus

{'Precision': 0.1,
 'Recall': 0.1,
 'f1 score': 0.10000000000000002,
 'Accuracy': 0.8604651162790697,
 'Balanced Accuracy': 0.5121848739495799,
 'Average Precision': 0.1363386518400739,
 'ROC AUC Score': 0.6781512605042017}

## Deep Isolation Forest (DIF)

Let's start with all the default parameters except for the contamination parameter that will be set at the real contamination of the dataset

In [8]:
p=sum(y)/len(y)
dif=DIF(contamination=p)
dif.fit(X_train)

DIF(batch_size=1000, contamination=0.07751937984496124,
  device=device(type='cpu'), hidden_activation='tanh',
  hidden_neurons=[500, 100], max_samples=256, n_ensemble=50,
  n_estimators=6, random_state=None, representation_dim=20,
  skip_connection=False)

Testing on X_test concatenation of X_train and X_test training on X_train 

In [9]:
dif_perf,y_pred=performance(dif,X_test,y)
dif_perf

{'Precision': 0.0,
 'Recall': 0.0,
 'f1 score': 0.0,
 'Accuracy': 0.9224806201550387,
 'Balanced Accuracy': 0.5,
 'Average Precision': 0.07751937984496124,
 'ROC AUC Score': 0.5}

Testing on X

In [10]:
dif_perf,y_pred=performance(dif,X,y)
dif_perf

{'Precision': 0.0,
 'Recall': 0.0,
 'f1 score': 0.0,
 'Accuracy': 0.9224806201550387,
 'Balanced Accuracy': 0.5,
 'Average Precision': 0.07751937984496124,
 'ROC AUC Score': 0.5}

## INNE 

In [11]:
p=sum(y)/len(y)
inne=INNE(contamination=p)
inne.fit(X_train)

INNE(contamination=0.07751937984496124, max_samples='auto', n_estimators=200,
   random_state=None)

Testing on X_test

In [12]:
dif_perf,y_pred=performance(inne,X_test,y)
dif_perf

{'Precision': 0.07142857142857142,
 'Recall': 0.1,
 'f1 score': 0.08333333333333333,
 'Accuracy': 0.8294573643410853,
 'Balanced Accuracy': 0.4953781512605042,
 'Average Precision': 0.07691029900332226,
 'ROC AUC Score': 0.4953781512605042}

Testing on X

In [13]:
dif_perf,y_pred=performance(inne,X,y)
dif_perf

{'Precision': 0.07692307692307693,
 'Recall': 0.1,
 'f1 score': 0.08695652173913043,
 'Accuracy': 0.8372093023255814,
 'Balanced Accuracy': 0.4995798319327731,
 'Average Precision': 0.07745974955277281,
 'ROC AUC Score': 0.49957983193277317}

## AutoEncoder

In the AutoEncoder model we have to reduce the dimensionality of the dataset in the hidden layer, so we have to pass the dimensions of each layer in the AutoEncoder in the `hidden_neurons` parameter.



In [6]:
p=sum(y)/len(y)
ae=AutoEncoder(contamination=p,hidden_neurons=[X_train.shape[1],7,7,X_train.shape[1]])
ae.fit(X_train)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 13)                182       
                                                                 
 dropout (Dropout)           (None, 13)                0         
                                                                 
 dense_1 (Dense)             (None, 13)                182       
                                                                 
 dropout_1 (Dropout)         (None, 13)                0         
                                                                 
 dense_2 (Dense)             (None, 13)                182       
                                                                 
 dropout_2 (Dropout)         (None, 13)                0         
                                                                 
 dense_3 (Dense)             (None, 7)                 9

2024-02-09 15:57:08.377740: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


None
Epoch 1/100
4/4 [==============================] - 1s 53ms/step - loss: 2.7339 - val_loss: 3.0937
Epoch 2/100
4/4 [==============================] - 0s 7ms/step - loss: 2.6257 - val_loss: 3.0238
Epoch 3/100
4/4 [==============================] - 0s 8ms/step - loss: 2.5882 - val_loss: 2.9588
Epoch 4/100
4/4 [==============================] - 0s 12ms/step - loss: 2.4714 - val_loss: 2.8976
Epoch 5/100
4/4 [==============================] - 0s 13ms/step - loss: 2.4530 - val_loss: 2.8431
Epoch 6/100
4/4 [==============================] - 0s 17ms/step - loss: 2.3420 - val_loss: 2.7954
Epoch 7/100
4/4 [==============================] - 0s 8ms/step - loss: 2.2713 - val_loss: 2.7528
Epoch 8/100
4/4 [==============================] - 0s 12ms/step - loss: 2.2597 - val_loss: 2.7141
Epoch 9/100
4/4 [==============================] - 0s 12ms/step - loss: 2.2084 - val_loss: 2.6769
Epoch 10/100
4/4 [==============================] - 0s 7ms/step - loss: 2.1877 - val_loss: 2.6432
Epoch 11/100
4/4 [

AutoEncoder(batch_size=32, contamination=0.07751937984496124,
      dropout_rate=0.2, epochs=100, hidden_activation='relu',
      hidden_neurons=[13, 7, 7, 13], l2_regularizer=0.1,
      loss=<function mean_squared_error at 0x7f3a1f5002c0>,
      optimizer='adam', output_activation='sigmoid', preprocessing=True,
      random_state=None, validation_size=0.1, verbose=1)

Testing on X_test

In [7]:
dif_perf,y_pred=performance(ae,X_test,y)
dif_perf

5/5 [==============================] - 0s 2ms/step


{'Precision': 0.2,
 'Recall': 0.2,
 'f1 score': 0.20000000000000004,
 'Accuracy': 0.875968992248062,
 'Balanced Accuracy': 0.5663865546218487,
 'Average Precision': 0.102015503875969,
 'ROC AUC Score': 0.5663865546218487}

Testing on X

In [8]:
dif_perf,y_pred=performance(ae,X,y)
dif_perf

5/5 [==============================] - 0s 3ms/step


{'Precision': 0.1111111111111111,
 'Recall': 0.1,
 'f1 score': 0.10526315789473685,
 'Accuracy': 0.8682170542635659,
 'Balanced Accuracy': 0.5163865546218488,
 'Average Precision': 0.08087855297157623,
 'ROC AUC Score': 0.5163865546218488}

## SUOD